In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow.keras import models,layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten
from sklearn.metrics import classification_report,confusion_matrix

## loading dataset

In [ ]:
#main_dir = '/content/drive/MyDrive/'
main_dir = '../input/plant-seedlings-classification'
train_dir = os.path.join(main_dir,"train")
test_dir = os.path.join(main_dir,"test")

In [ ]:

labels = sorted(os.listdir(train_dir))
print(labels)

In [ ]:
num_classes = len(labels)
num_classes

In [ ]:
classes = list(labels)

In [ ]:
classes

## preprocessing

In [ ]:
batch_size = 32
img_height = 224
img_width = 224

In [ ]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator 


In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1./255.,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2, 
    shear_range = 0.2, 
    zoom_range = 0.2, 
    validation_split=0.20,
    horizontal_flip = True
    )

In [ ]:
train_generator = train_datagen.flow_from_directory(train_dir, 
                                                    batch_size = 20, 
                                                    subset = 'training',
                                                    target_size = (img_height, img_width))




In [ ]:
validation_generator = train_datagen.flow_from_directory(train_dir, 
                                                        batch_size = 20,
                                                         subset='validation',
                                                        target_size = (img_height, img_width))

## VGG


In [ ]:
from keras.applications.vgg16 import preprocess_input,decode_predictions,VGG16
base_model = VGG16( input_shape = (img_height,img_width, 3), 
include_top = False, 
weights = 'imagenet')

##### 18 layers 


In [ ]:
# for i, layer in enumerate(base_model.layers):
#     print(i, layer.name, layer.trainable)

for layer in base_model.layers[:15]:
    layer.trainable = False

# for i, layer in enumerate(base_model.layers):
#     print(i, layer.name, layer.trainable)

In [ ]:
x = base_model.output
x = layers.Flatten()(x)
x = Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x) 
x = Dense(256, activation='relu')(x)
x = Dense(12, activation='softmax')(x)
vgg_model = models.Model(inputs=base_model.input, outputs=x)

In [ ]:
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
lr_reduce = ReduceLROnPlateau(monitor='val_accuracy', factor=0.6, patience=5, verbose=1, mode='max', min_lr=5e-5)

checkpoint = ModelCheckpoint('vgg.h5', monitor= 'val_accuracy', mode= 'max', save_best_only = True, verbose= 1)

early_stop=EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=5)
callbacklist = [lr_reduce,checkpoint,early_stop]

In [ ]:
from tensorflow.keras import layers, models, Model, optimizers
learning_rate= 5e-5
vgg_model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(lr=learning_rate), metrics=["accuracy"])

vgghistory = vgg_model.fit_generator(
    train_generator, 
    validation_data=validation_generator, 
    epochs=10, 
    shuffle=True, 
    callbacks =callbacklist,
    verbose=1)

In [ ]:
acc = vgghistory.history['accuracy']
val_acc = vgghistory.history['val_accuracy']

loss = vgghistory.history['loss']
val_loss = vgghistory.history['val_loss']

epochs_range = range(10)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
#vgg_model.save('vgg.h5')

In [ ]:
y_pred1 = vgg_model.predict(validation_generator)
y_pred1 = np.argmax(y_pred1, axis=1)
y_pred1.sort()

In [ ]:
cm1 = confusion_matrix(validation_generator.classes, y_pred1)


In [ ]:
import itertools
def plot_confusion_matrix(cm, classes, normalize=True, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.figure(figsize=(10,10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        cm = np.around(cm, decimals=2)
        cm[np.isnan(cm)] = 0.0
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
plot_confusion_matrix(cm1, classes, title='Confusion Matrix')

In [ ]:
print(classification_report(validation_generator.classes, y_pred1,target_names = classes))

## RESNET

In [ ]:

from tensorflow.keras.applications.resnet50 import ResNet50
base_model1 = ResNet50( input_shape = (img_height,img_width, 3),include_top = False, weights = 'imagenet')

#### 174 layers

In [ ]:
# for i, layer in enumerate(base_model1.layers):
#     print(i, layer.name, layer.trainable)
    

for layer in base_model1.layers[:15]:
    layer.trainable = False

In [ ]:
x1 = base_model1.output
x1 = layers.Flatten()(x1)
x1 = Dense(512, activation='relu')(x1)
x1 = layers.Dropout(0.5)(x1) 
x1 = Dense(256, activation='relu')(x1)

x1 = Dense(12, activation='softmax')(x1)
resnet_model = models.Model(inputs=base_model1.input, outputs=x1)


In [ ]:
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
lr_reduce = ReduceLROnPlateau(monitor='val_accuracy', factor=0.05, patience=5, verbose=1, mode='max', min_lr=0.000002)

checkpoint = ModelCheckpoint('resnet.h5', monitor= 'val_accuracy', mode= 'max', save_best_only = True, verbose= 1)

early_stop=EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=5)
callbacklist = [lr_reduce,checkpoint,early_stop]

In [ ]:
from tensorflow.keras import layers, models, Model, optimizers
learning_rate= 5e-5
resnet_model.compile(
    loss="categorical_crossentropy", 
    optimizer=optimizers.Adam(learning_rate=learning_rate), metrics=["accuracy"])


In [ ]:
resnethistory = resnet_model.fit_generator(
    train_generator, 
    validation_data=validation_generator, 
    epochs=10, 
    shuffle=True, 
    callbacks = callbacklist,
    verbose=1)


In [ ]:
acc = resnethistory.history['accuracy']
val_acc = resnethistory.history['val_accuracy']

loss = resnethistory.history['loss']
val_loss = resnethistory.history['val_loss']

epochs_range = range(10)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
y_pred2 = resnet_model.predict(validation_generator)
y_pred2 = np.argmax(y_pred2, axis=1)
y_pred2.sort()


In [ ]:
cm2 = confusion_matrix(validation_generator.classes, y_pred2)


In [ ]:
plot_confusion_matrix(cm2, classes, title='Confusion Matrix')


In [ ]:
print(classification_report(validation_generator.classes, y_pred2,target_names = classes))


In [ ]:
#resnet_model.save('resnet.h5')

## Inception

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
base_model2 = InceptionV3(input_shape = (img_height,img_width, 3), include_top = False, weights = 'imagenet')


#### 310 layers

In [ ]:
# for i, layer in enumerate(base_model2.layers):
#     print(i, layer.name, layer.trainable)

for layer in base_model2.layers[:15]:
    layer.trainable = False

# for i, layer in enumerate(base_model2.layers):
#     print(i, layer.name, layer.trainable)
#310 layers

In [ ]:
x2 = base_model2.output
x2 = layers.Flatten()(x2)
x2 = Dense(512, activation='relu')(x2)
x2 = layers.Dropout(0.5)(x2) 
x2 = Dense(256, activation='relu')(x2)
x2 = Dense(12, activation='softmax')(x2) 
inc_model = models.Model(inputs=base_model2.input, outputs=x2)


In [ ]:
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
lr_reduce = ReduceLROnPlateau(monitor='val_accuracy', factor=0.6, patience=5, verbose=1, mode='max', min_lr=5e-5)

checkpoint = ModelCheckpoint('./inc.h5', monitor= 'val_accuracy', mode= 'max', save_best_only = True, verbose= 1)
early_stop=EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=5)
callbacklist = [lr_reduce,early_stop,checkpoint]

In [ ]:
from tensorflow.keras import layers, models, Model, optimizers
learning_rate= 5e-5
inc_model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(learning_rate=learning_rate), metrics=["accuracy"])



In [ ]:
inchistory = inc_model.fit_generator(
    train_generator, 
    validation_data=validation_generator, 
    epochs=10, 
    shuffle=True, 
    callbacks = callbacklist,
    verbose=1)


In [ ]:
acc = inchistory.history['accuracy']
val_acc = inchistory.history['val_accuracy']

loss = inchistory.history['loss']
val_loss = inchistory.history['val_loss']

epochs_range = range(10)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()



In [ ]:
y_pred3 = inc_model.predict(validation_generator)
y_pred3 = np.argmax(y_pred3, axis=1)
y_pred3.sort()

In [ ]:
cm3 = confusion_matrix(validation_generator.classes, y_pred3)


In [ ]:
plot_confusion_matrix(cm3, classes, title='Confusion Matrix')

In [ ]:
print(classification_report(validation_generator.classes, y_pred3,target_names = classes))

In [ ]:
#inc_model.save('inc.h5')